# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
exported_cities_data = ('../output_data/cities_data.csv')
Weather_Results_df = pd.read_csv(exported_cities_data)
Weather_Results_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,73.18,72,1,17.87,PF,1626982231
1,berlevag,70.8578,29.0864,44.55,81,75,29.93,NO,1626981685
2,antalaha,-14.9003,50.2788,68.02,92,79,7.05,MG,1626982232
3,jamestown,42.0970,-79.2353,72.75,50,1,5.99,US,1626982221
4,hobyo,5.3505,48.5268,77.86,84,6,32.93,SO,1626981698


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
#Store 'Lat' and 'Lng' into  locations 
Locations = Weather_Results_df[['Lat', 'Lng']].astype(float)

#Store 'Humidity'
Humidity = Weather_Results_df['Humidity'].astype(float)

In [5]:
#Create a figure
fig = gmaps.figure(zoom_level = 2, center = (25, -5))

#Create a humidity layer
Humidity_layer = gmaps.heatmap_layer(Locations, weights=Humidity, dissipating=False, max_intensity=170, point_radius = 4)

#Add the humidity layer to the figure
fig.add_layer(Humidity_layer)

#Show the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Creating Ideal Vacation DataFrame (filtered by my own preferences)
Ideal_Vacation_df = Weather_Results_df.loc[(Weather_Results_df['Max Temp'] >= 68) & 
                                           (Weather_Results_df['Max Temp'] <= 75) & 
                                           (Weather_Results_df['Cloudiness'] < 5) &
                                           (Weather_Results_df['Humidity'] <60) & 
                                           (Weather_Results_df['Wind Speed'] <5),
                                           :]
Ideal_Vacation_df = Ideal_Vacation_df.dropna(how='any')
Ideal_Vacation_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,taolanaro,-32.9468,-60.6393,72.73,40,0,3.00,AR,1626982236
142,kavaratti,42.2506,-75.8330,73.06,56,1,3.00,US,1626981743
246,champerico,47.5001,-120.5015,73.98,26,0,4.25,US,1626982197
327,nenjiang,50.2499,-117.8023,73.74,31,2,2.30,CA,1626982344
412,nanortalik,38.3906,35.4922,74.55,44,0,4.70,TR,1626982374
415,quatre cocos,44.0582,-121.3153,72.07,24,1,1.99,US,1626982210
466,morehead,-26.6167,-54.1333,71.37,43,0,2.62,AR,1626982393


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = Ideal_Vacation_df

In [8]:
#Adding 'Hotel Name' column to my hotel_df DataFrame
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,taolanaro,-32.9468,-60.6393,72.73,40,0,3.00,AR,1626982236,
142,kavaratti,42.2506,-75.8330,73.06,56,1,3.00,US,1626981743,
246,champerico,47.5001,-120.5015,73.98,26,0,4.25,US,1626982197,
327,nenjiang,50.2499,-117.8023,73.74,31,2,2.30,CA,1626982344,
412,nanortalik,38.3906,35.4922,74.55,44,0,4.70,TR,1626982374,
415,quatre cocos,44.0582,-121.3153,72.07,24,1,1.99,US,1626982210,
466,morehead,-26.6167,-54.1333,71.37,43,0,2.62,AR,1626982393,


In [12]:
#Base URL 
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#Populate hotel name for each city in the hotel_df DataFrame
hotel_list = []

#Creatign for loop to get hotel name for each city in the hotel_df DataFrame
for index in hotel_df.index:
    Lat_and_Lng = str(hotel_df['Lat'][index]) + ',' + str(hotel_df['Lng'][index])
    
    #Set parameters
    target_coordinates = Lat_and_Lng
    target_type = "hotel"
    radius = 5000

    #Rewrite params dict
    params = {
        'location': target_coordinates,
        'types': target_type,
        'radius': radius,
        'key': g_key
    }
    #API response in json format
    response = requests.get(base_url, params=params).json()
    
    #adding hottel names to the hotel_list
    hotel_list.append(response['results'][1]['name'])

#Populating hotel names in the 'Hotel Name' column    
hotel_df['Hotel Name'] = hotel_list

#Displaying hotel_df to see if Hotel Names are added to the DataFrame
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,taolanaro,-32.9468,-60.6393,72.73,40,0,3.00,AR,1626982236,"Holiday Inn Rosario, an IHG Hotel"
142,kavaratti,42.2506,-75.8330,73.06,56,1,3.00,US,1626981743,Chenango Valley State Park
246,champerico,47.5001,-120.5015,73.98,26,0,4.25,US,1626982197,Cashmere High School
327,nenjiang,50.2499,-117.8023,73.74,31,2,2.30,CA,1626982344,CIBC Branch with ATM
412,nanortalik,38.3906,35.4922,74.55,44,0,4.70,TR,1626982374,Barış Büfe 2 Tekel
415,quatre cocos,44.0582,-121.3153,72.07,24,1,1.99,US,1626982210,Fairfield Inn & Suites by Marriott Bend Downtown
466,morehead,-26.6167,-54.1333,71.37,43,0,2.62,AR,1626982393,Banco Macro Sucursal 13 San Pedro


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
#Create a figure
fig = gmaps.figure(zoom_level = 2, center = (25, -5))

#Create Hotel marker layer
Hotel_layer = gmaps.marker_layer(
    locations, info_box_content = hotel_info
)
#Add the 'humidity' and 'hotel' layers to the figure
fig.add_layer(Humidity_layer)
fig.add_layer(Hotel_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))